In [73]:
import pandas as pd
import numpy as np
import requests
import urllib.parse
import os

### Get lat/lon

In [124]:
antwerp_streets = pd.read_csv('../data/straatnamen_antwerpen.csv')
straten_centrum = antwerp_streets[antwerp_streets['postcode']==2000][['LSTRNM', 'postcode', 'DISTRICT']].drop_duplicates().reset_index(drop=True)
straten_centrum['postcode'] = straten_centrum['postcode'].astype('str')
straten_centrum.dropna(inplace=True)
straten_centrum.drop(index=501, inplace=True)

In [126]:
def get_lat_lon(address):
    url = "https://nominatim.openstreetmap.org"

    params = {
        'q': address,
        'format': 'json'
    }
    
    response = requests.get(url, params=params).json()
    if len(response) > 0:
        return response[0]['lat'], response[0]['lon']
    return np.nan, np.nan

In [129]:
lat=[]
lon=[]

for i, d in straten_centrum.iterrows():
    print(i/len(straten_centrum))
    llat, llon = get_lat_lon(' '.join(d.values))
    lat.append(llat)
    lon.append(llon)

0.0
0.001976284584980237
0.003952569169960474
0.005928853754940711
0.007905138339920948
0.009881422924901186
0.011857707509881422
0.01383399209486166
0.015810276679841896
0.017786561264822136
0.019762845849802372
0.021739130434782608
0.023715415019762844
0.025691699604743084
0.02766798418972332
0.029644268774703556
0.03162055335968379
0.03359683794466403
0.03557312252964427
0.037549407114624504
0.039525691699604744
0.041501976284584984
0.043478260869565216
0.045454545454545456
0.04743083003952569
0.04940711462450593
0.05138339920948617
0.0533596837944664
0.05533596837944664
0.05731225296442688
0.05928853754940711
0.06126482213438735
0.06324110671936758
0.06521739130434782
0.06719367588932806
0.0691699604743083
0.07114624505928854
0.07312252964426877
0.07509881422924901
0.07707509881422925
0.07905138339920949
0.08102766798418973
0.08300395256916997
0.08498023715415019
0.08695652173913043
0.08893280632411067
0.09090909090909091
0.09288537549407115
0.09486166007905138
0.09683794466403162


In [203]:
straten_centrum['lat'] = lat
straten_centrum['lon'] = lon
straten_centrum.head()


straten_centrum.to_csv('../data/straatnamen_antwerpen_latlon.csv')

### Weather

In [204]:
straten_centrum = pd.read_csv('../data/straatnamen_antwerpen_latlon.csv', index_col=0)

In [151]:
BASE_URI = 'https://api.openweathermap.org'

In [157]:
def get_forecasts(lat, lon):

    params = {
        'lat': lat,
        'lon': lon,
        'appid': os.getenv('API_KEY'),
        'units':'metric'
    }
    
    url = urllib.parse.urljoin(BASE_URI, '/data/2.5/forecast')
    
    #url = BASE_URI + '/data/2.5/forecast'
    
    forecasts = requests.get(url, params=params).json()['list']
    
    return forecasts

In [164]:
def get_wind(dict, address):
    new_dict = dict['wind']
    new_dict['address']= address
    new_dict['time']= dict['dt_txt']
    return new_dict

In [187]:
ls = []
def get_df(df):
    for i, row in df.iterrows():
        forecast = get_forecasts(row['lat'], row['lon'])
        for j in forecast:
            new_dict = get_wind(j, row['LSTRNM'] + ' ' + str(row['postcode']) + ' ' + row['DISTRICT'])
            ls.append(new_dict)

In [188]:
from more_itertools import sliced
CHUNK_SIZE = 60

index_slices = sliced(range(len(straten_centrum)), CHUNK_SIZE)

In [189]:
import time

for slice in index_slices:
    chunck = straten_centrum.iloc[slice]
    get_df(chunck)
    time.sleep(60)

KeyError: 'list'

In [192]:
all_streets_df = pd.DataFrame(ls)
all_streets_df.head()

,speed,deg,gust,address,time
0,2.44,176,5.12,Aalmoezenierstraat 2000 Antwerpen,2022-11-14 15:00:00
1,3.29,186,8.85,Aalmoezenierstraat 2000 Antwerpen,2022-11-14 18:00:00
2,3.58,194,10.24,Aalmoezenierstraat 2000 Antwerpen,2022-11-14 21:00:00
3,4.18,190,9.65,Aalmoezenierstraat 2000 Antwerpen,2022-11-15 00:00:00
4,4.17,198,9.52,Aalmoezenierstraat 2000 Antwerpen,2022-11-15 03:00:00


In [205]:
straten_centrum

,LSTRNM,postcode,DISTRICT,lat,lon,full_address
0,Aalmoezenierstraat,2000,Antwerpen,51.214127,4.400656,Aalmoezenierstraat 0 2000\n1 2000\n2...
1,Admiraal de Boisotstraat,2000,Antwerpen,51.206665,4.392862,Admiraal de Boisotstraat 0 2000\n1 2...
2,Adriaan Brouwerstraat,2000,Antwerpen,51.227570,4.403460,Adriaan Brouwerstraat 0 2000\n1 2000...
3,Allewaertstraat,2000,Antwerpen,51.207645,4.394302,Allewaertstraat 0 2000\n1 2000\n2 ...
4,Ambtmanstraat,2000,Antwerpen,51.222433,4.405923,Ambtmanstraat 0 2000\n1 2000\n2 ...
...,...,...,...,...,...,...
503,André Delvauxstraat,2000,Antwerpen,NaN,NaN,André Delvauxstraat 0 2000\n1 2000\n...
504,Henri Storckstraat,2000,Antwerpen,NaN,NaN,Henri Storckstraat 0 2000\n1 2000\n2...
505,Edith Kielstraat,2000,Antwerpen,NaN,NaN,Edith Kielstraat 0 2000\n1 2000\n2 ...
506,Nicole Van Goethemstraat,2000,Antwerpen,NaN,NaN,Nicole Van Goethemstraat 0 2000\n1 2...
